In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder


In [2]:
# filepath to your JSON file
filepath = "dataset.json"

# Read the JSON data into a DataFrame
df = pd.read_json(filepath)

In [3]:
df.shape

(1222, 2)

In [3]:
print(len(df['externalStatus'].unique()))
print(len(df['internalStatus'].unique()))

108
15


In [4]:
df['internalStatus'].unique()

array(['Port Out', 'Inbound Terminal', 'Port In', 'Departure', 'Arrival',
       'Gate In', 'Loaded on Vessel', 'Gate Out', 'On Rail', 'Off Rail',
       'Empty Return', 'In-transit', 'Outbound Terminal',
       'Empty Container Released', 'Unloaded on Vessel'], dtype=object)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a tokenizer for external status
tokenizer = Tokenizer(num_words=1000, oov_token="<UNK>")  # Adjust num_words as needed
tokenizer.fit_on_texts(df['externalStatus'])
external_status_sequences = tokenizer.texts_to_sequences(df['externalStatus'])
max_length = max(len(seq) for seq in external_status_sequences)
padded_sequences = keras.preprocessing.sequence.pad_sequences(external_status_sequences, maxlen=max_length, padding='post')


In [6]:
internal_status_encoded = pd.get_dummies(df['internalStatus'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, internal_status_encoded, test_size=0.2, random_state=42)


In [8]:
model = keras.Sequential([
    layers.Embedding(input_dim=tokenizer.num_words, output_dim=128, input_length=max_length),  # Adjust embedding dimensions as needed
    layers.GlobalAveragePooling1D(),
    layers.Dense(128, activation='relu'),  # Adjust hidden layer size and activation as needed
    layers.Dense(15, activation='softmax')  # Output layer with 15 units for 15 categories
])

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test)) 

Epoch 1/10
31/31 [==============================] - 1s 7ms/step - loss: 2.5534 - accuracy: 0.2825 - val_loss: 2.3604 - val_accuracy: 0.3592
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 2.0710 - accuracy: 0.4759 - val_loss: 2.0750 - val_accuracy: 0.4571
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 1.7463 - accuracy: 0.5077 - val_loss: 1.7468 - val_accuracy: 0.4612
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 1.4083 - accuracy: 0.5148 - val_loss: 1.4621 - val_accuracy: 0.6898
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1443 - accuracy: 0.7236 - val_loss: 1.2372 - val_accuracy: 0.7020
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9356 - accuracy: 0.7912 - val_loss: 1.0415 - val_accuracy: 0.7796
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 0.7502 - accuracy: 0.8516 - val_loss: 0.8379 - val_accuracy: 0.8082
Epoch 8/10
31/31 [==

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

8/8 [==============================] - 0s 1ms/step - loss: 0.4106 - accuracy: 0.9224
Test accuracy: 0.922448992729187


In [11]:
new_status = "Discharge (Vessel name : COSCO INDONESIA)"
new_sequence = tokenizer.texts_to_sequences([new_status])[0]
new_sequence = keras.preprocessing.sequence.pad_sequences([new_sequence], maxlen=max_length, padding='post')
prediction = model.predict(new_sequence)
predicted_class = internal_status_encoded.columns[prediction.argmax()] 
print('Predicted internal status:', predicted_class)


1/1 [==============================] - 0s 70ms/step
Predicted internal status: Departure


In [12]:
model.save("my_model.h5")

C:\Users\rajes\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
